Approach 1: treat all rows independently
Approach 2: keep posts and comments separate
Approach 3: combine posts and comments

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
import nltk
from nltk.corpus import stopwords
import spacy
from gensim.models import CoherenceModel
import pyLDAvis.gensim
import pyLDAvis
import warnings
import matplotlib.pyplot as plt

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
file_path = 'C:/Users/user/OneDrive - Oulun yliopisto/Documents/suomi24/Data/suomi24.csv'
df = pd.read_csv(file_path)
df

,msg_type,datetime,title,thread_id,comment_id,topic_name_top,topic_name_leaf,thread_text,processed_text,topic,probability
0,thread_start,2001-01-18 17:18:00,Questions about Estonia,89742,0,Society,# The world's going on #,- Why did an important piece of evidence (from...,important piece evidence front gate fall depth...,-1,important piece evidence front gate fall depth...
1,comment,2001-01-19 01:28:00,Questions about Estonia,89742,386913,Society,# The world's going on #,I think they had other things to do. - First o...,think thing first kin demanded sepulchre would...,-1,think thing first kin demanded sepulchre would...
2,comment,2001-01-19 06:21:00,Questions about Estonia,89742,386914,Society,# The world's going on #,"- Not the bow gate, but its SARANA- &gt; a str...",bow gate sarana strange coincidence fell resea...,-1,bow gate sarana gt strange coincidence fell re...
3,comment,2001-01-19 16:27:00,Questions about Estonia,89742,386915,Society,# The world's going on #,I'll tell you what's going on. I'll tell you w...,ill tell whats going ill tell whatll news,350,ill tell whats going ill tell whatll news
4,comment,2001-01-20 12:49:00,Questions about Estonia,89742,386916,Society,# The world's going on #,Let the cause of the sinking of any ship be li...,let cause sinking ship lifted away wreckage le...,-1,let cause sinking ship lifted away wreckage le...
...,...,...,...,...,...,...,...,...,...,...,...
273299,comment,2020-12-31 17:08:00,Thai rice dogs,16711386,105838676,Tourism,Pattaya,"as if these omega- ""men"" were taken there. The...",omega men taken comedian claim nasumarttaamal ...,-1,omega men taken comedian claim nasumarttaamal ...
273300,thread_start,2020-12-30 17:58:00,Surveillance camera,16711801,0,Home and construction,Lighting and electrical works,Surveillance camera Is there a human next outs...,surveillance camera human next outside camera,55,surveillance camera human next outside camera g
273301,comment,2020-12-31 06:34:00,Surveillance camera,16711801,105829873,Home and construction,Lighting and electrical works,Pretty sure the camera will follow you. Put th...,pretty sure camera follow put security camera ...,-1,pretty sure camera follow put security camera ...
273302,comment,2020-12-31 13:27:00,Surveillance camera,16711801,105835154,Home and construction,Lighting and electrical works,please see online if you have Internet access ...,please see online internet access baron kepele...,136,please see online internet access baron kepele...


# Preprocess

In [33]:
# Basic text preprocessing function
stop_words = set(stopwords.words('english'))
stop_words.add('s')
stop_words.add('ve')
stop_words.add('re')
stop_words.add('m')
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()  # Convert to lower case
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\b\w{1,2}\b', '', text) # Remove single and double character words
    text = text.lower()  # Convert to lower case
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Apply preprocessing to the 'thread_text' column
df['processed_text'] = df['thread_text'].astype(str).apply(preprocess_text)


In [34]:
df['processed_text'][0]

' why did an important piece of evidence from the front gate  fall down into the depths of the sea from the helicopter that carried it questions about estonia  why isnt estonia the only ship in the world that immediately held up the peace of the grave  why did the border guard prevent investigations by an independent tv company before the area was protected  why is the socalled burial peace repeatedly invoked even though most of the relatives wanted to study the matter  why only one theory of sinking even though it is very controversial  flashed into the public domain as truth  didnt anyone get saved from the bridge of command even though they were in the advantage  why were no other images given to the public when they had a couple of stormy images'

In [35]:
# Define a function to filter relevant rows
def is_relevant(text):
    # Example criterion: contains certain keywords
    keywords = ['camera', 'public', 'privacy', 'cctv', 'surveillance', 'security', 'safety', 'concern', 'protect', 'monitoring', 'footage', 'personal', 'cameras', 'concerns', 'protection', 'monitor']
    return any(keyword in text for keyword in keywords)

# Apply the function to filter rows
df_relevant = df[df['processed_text'].apply(is_relevant)]

In [36]:
df_relevant

,msg_type,datetime,title,thread_id,comment_id,topic_name_top,topic_name_leaf,thread_text,processed_text
0,thread_start,2001-01-18 17:18:00,Questions about Estonia,89742,0,Society,# The world's going on #,- Why did an important piece of evidence (from...,why did an important piece of evidence from t...
15,comment,2001-01-22 11:16:00,Questions about Estonia,89742,386927,Society,# The world's going on #,intutorytutkimus. fi and there MV Estonia. The...,intutorytutkimus fi and there mv estonia the f...
27,thread_start,2001-01-19 12:15:00,automatic control is just money laundering,32552,0,Vehicles and transport,General information on driving,car thieves and so on. But when speed restrict...,car thieves and so on but when speed restricti...
28,comment,2001-01-19 12:59:00,automatic control is just money laundering,32552,162166,Vehicles and transport,General information on driving,how can you teach your children to walk throug...,how can you teach your children to walk throug...
31,comment,2001-01-19 17:05:00,automatic control is just money laundering,32552,162170,Vehicles and transport,General information on driving,even though I don't have a radar detector or e...,even though i dont have a radar detector or ev...
...,...,...,...,...,...,...,...,...,...
274498,comment,2020-12-31 10:33:44,Thai rice dogs,16711386,105832145,Tourism,Pattaya,"quiet area, but now I have a house at three me...",quiet area but now i have a house at three met...
274502,comment,2020-12-31 13:32:47,Thai rice dogs,16711386,105835231,Tourism,Pattaya,At that time they were expensive devices. Well...,at that time they were expensive devices well ...
274507,comment,2020-12-31 14:34:39,Thai rice dogs,16711386,105836278,Tourism,Pattaya,"someone doesn't match. . Earlier, you told me ...",someone doesnt match earlier you told me you ...
274515,thread_start,2020-12-30 17:58:06,Surveillance camera,16711801,0,Home and construction,Lighting and electrical works,Surveillance camera Is there a human next outs...,surveillance camera is there a human next outs...


Topic Modeling

In [4]:
def sent_to_words(texts):
    for text in texts:
        yield(simple_preprocess(str(text),deacc=True))  # deacc=True removes the punctuation marks
        
data_words=list(sent_to_words(df['processed_text']))
# data_words=list(sent_to_words(df_relevant['processed_text']))

In [5]:
data_words

[['important',
  'piece',
  'evidence',
  'front',
  'gate',
  'fall',
  'depth',
  'sea',
  'helicopter',
  'carried',
  'question',
  'estonia',
  'isnt',
  'estonia',
  'ship',
  'world',
  'immediately',
  'held',
  'peace',
  'grave',
  'border',
  'guard',
  'prevent',
  'investigation',
  'independent',
  'company',
  'area',
  'protected',
  'socalled',
  'burial',
  'peace',
  'repeatedly',
  'invoked',
  'even',
  'though',
  'relative',
  'wanted',
  'study',
  'matter',
  'one',
  'theory',
  'sinking',
  'even',
  'though',
  'controversial',
  'flashed',
  'public',
  'domain',
  'truth',
  'didnt',
  'anyone',
  'get',
  'saved',
  'bridge',
  'command',
  'even',
  'though',
  'advantage',
  'image',
  'given',
  'public',
  'couple',
  'stormy',
  'image'],
 ['think',
  'thing',
  'first',
  'kin',
  'demanded',
  'sepulchre',
  'would',
  'rejected',
  'independent',
  'picture',
  'bottom',
  'wreck',
  'taken',
  'lightning',
  'light',
  'fear',
  'scuba',
  'dive'

In [6]:
stop_words=stopwords.words('english')
stop_words.append('s')
stop_words.append('ve')
stop_words.append('re')
stop_words.append('m')
print(stop_words)
# remove stopwords
def remove_stopwords(texts):
    return [[word for word in text if word not in stop_words] for text in texts]

data_words_nostops=remove_stopwords(data_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
len(data_words_nostops)

273304

In [8]:
data_words_nostops

[['important',
  'piece',
  'evidence',
  'front',
  'gate',
  'fall',
  'depth',
  'sea',
  'helicopter',
  'carried',
  'question',
  'estonia',
  'isnt',
  'estonia',
  'ship',
  'world',
  'immediately',
  'held',
  'peace',
  'grave',
  'border',
  'guard',
  'prevent',
  'investigation',
  'independent',
  'company',
  'area',
  'protected',
  'socalled',
  'burial',
  'peace',
  'repeatedly',
  'invoked',
  'even',
  'though',
  'relative',
  'wanted',
  'study',
  'matter',
  'one',
  'theory',
  'sinking',
  'even',
  'though',
  'controversial',
  'flashed',
  'public',
  'domain',
  'truth',
  'didnt',
  'anyone',
  'get',
  'saved',
  'bridge',
  'command',
  'even',
  'though',
  'advantage',
  'image',
  'given',
  'public',
  'couple',
  'stormy',
  'image'],
 ['think',
  'thing',
  'first',
  'kin',
  'demanded',
  'sepulchre',
  'would',
  'rejected',
  'independent',
  'picture',
  'bottom',
  'wreck',
  'taken',
  'lightning',
  'light',
  'fear',
  'scuba',
  'dive'

In [9]:
# Applying Bigrams and trigrams 
bigram=gensim.models.Phrases(data_words_nostops,min_count=5,threshold=80)
trigram=gensim.models.Phrases(bigram[data_words_nostops],threshold=80)

bigram_mod=gensim.models.phrases.Phraser(bigram)
trigram_mod=gensim.models.phrases.Phraser(trigram)

def make_bigram(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigram(texts):
    return [trigram_mod[doc] for doc in texts]

data_words_nostops_bigrams=make_bigram(data_words_nostops)
data_words_nostops_trigrams=make_bigram(data_words_nostops_bigrams)

In [10]:
bigram_phrases = [phrase for phrase, score in bigram.vocab.items() if '_' in phrase]

In [11]:
bigram_phrases

['important_piece',
 'piece_evidence',
 'evidence_front',
 'front_gate',
 'gate_fall',
 'fall_depth',
 'depth_sea',
 'sea_helicopter',
 'helicopter_carried',
 'carried_question',
 'question_estonia',
 'estonia_isnt',
 'isnt_estonia',
 'estonia_ship',
 'ship_world',
 'world_immediately',
 'immediately_held',
 'held_peace',
 'peace_grave',
 'grave_border',
 'border_guard',
 'guard_prevent',
 'prevent_investigation',
 'investigation_independent',
 'independent_company',
 'company_area',
 'area_protected',
 'protected_socalled',
 'socalled_burial',
 'burial_peace',
 'peace_repeatedly',
 'repeatedly_invoked',
 'invoked_even',
 'even_though',
 'though_relative',
 'relative_wanted',
 'wanted_study',
 'study_matter',
 'matter_one',
 'one_theory',
 'theory_sinking',
 'sinking_even',
 'though_controversial',
 'controversial_flashed',
 'flashed_public',
 'public_domain',
 'domain_truth',
 'truth_didnt',
 'didnt_anyone',
 'anyone_get',
 'get_saved',
 'saved_bridge',
 'bridge_command',
 'command_ev

In [12]:
trigram_phrases = [phrase for phrase, score in trigram.vocab.items() if '_' in phrase]

In [13]:
trigram_phrases

['important_piece',
 'piece_evidence',
 'evidence_front',
 'front_gate',
 'gate_fall',
 'fall_depth',
 'depth_sea',
 'sea_helicopter',
 'helicopter_carried',
 'carried_question',
 'question_estonia',
 'estonia_isnt',
 'isnt_estonia',
 'estonia_ship',
 'ship_world',
 'world_immediately',
 'immediately_held',
 'held_peace',
 'peace_grave',
 'grave_border',
 'border_guard',
 'guard_prevent',
 'prevent_investigation',
 'investigation_independent',
 'independent_company',
 'company_area',
 'area_protected',
 'protected_socalled',
 'burial_peace',
 'socalled_burial_peace',
 'burial_peace_repeatedly',
 'repeatedly_invoked',
 'invoked_even',
 'even_though',
 'though_relative',
 'relative_wanted',
 'wanted_study',
 'study_matter',
 'matter_one',
 'one_theory',
 'theory_sinking',
 'sinking_even',
 'though_controversial',
 'controversial_flashed',
 'public_domain',
 'flashed_public_domain',
 'public_domain_truth',
 'truth_didnt',
 'didnt_anyone',
 'anyone_get',
 'get_saved',
 'saved_bridge',
 'br

In [14]:
len(data_words_nostops_bigrams)

273304

In [15]:
len(data_words_nostops_trigrams)

273304

In [16]:
data_words_nostops_bigrams

[['important',
  'piece',
  'evidence',
  'front',
  'gate',
  'fall',
  'depth',
  'sea',
  'helicopter',
  'carried',
  'question',
  'estonia',
  'isnt',
  'estonia',
  'ship',
  'world',
  'immediately',
  'held',
  'peace',
  'grave',
  'border',
  'guard',
  'prevent',
  'investigation',
  'independent',
  'company',
  'area',
  'protected',
  'socalled',
  'burial_peace',
  'repeatedly',
  'invoked',
  'even',
  'though',
  'relative',
  'wanted',
  'study',
  'matter',
  'one',
  'theory',
  'sinking',
  'even',
  'though',
  'controversial',
  'flashed',
  'public_domain',
  'truth',
  'didnt',
  'anyone',
  'get',
  'saved',
  'bridge',
  'command',
  'even',
  'though',
  'advantage',
  'image',
  'given',
  'public',
  'couple',
  'stormy',
  'image'],
 ['think',
  'thing',
  'first',
  'kin',
  'demanded',
  'sepulchre',
  'would',
  'rejected',
  'independent',
  'picture',
  'bottom',
  'wreck',
  'taken',
  'lightning',
  'light',
  'fear',
  'scuba',
  'dive',
  'invas

In [17]:
nlp=spacy.load('en_core_web_sm',disable=['parser','ner'])

def lemmatize(texts,allowed_postags=['NOUN','ADJ','VERB','ADV']):
    texts_out=[]
    for sent in texts:
        doc=nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        
    return texts_out

In [18]:
data_lemmatized=lemmatize(data_words_nostops_bigrams,allowed_postags=['NOUN','ADJ','VERB','ADV'] )

In [19]:
data_lemmatized

[['important',
  'piece',
  'evidence',
  'depth',
  'sea',
  'helicopter',
  'carry',
  'question',
  'ship',
  'world',
  'immediately',
  'hold',
  'peace',
  'grave',
  'border',
  'guard',
  'prevent',
  'investigation',
  'independent',
  'company',
  'area',
  'protect',
  'socalle',
  'burial_peace',
  'repeatedly',
  'invoke',
  'even',
  'relative',
  'want',
  'study',
  'matter',
  'theory',
  'sink',
  'even',
  'controversial',
  'flash',
  'public_domain',
  'truth',
  'get',
  'save',
  'bridge',
  'command',
  'even',
  'advantage',
  'image',
  'give',
  'public',
  'couple',
  'stormy',
  'image'],
 ['think',
  'thing',
  'demand',
  'reject',
  'independent',
  'picture',
  'bottom',
  'wreck',
  'take',
  'lightning',
  'light',
  'fear',
  'dive',
  'invasion',
  'wreckage',
  'able',
  'save',
  'accident',
  'first',
  'investigate',
  'authority',
  'television',
  'company',
  'wonder',
  'gate',
  'raise',
  'thing',
  'stormy',
  'thing',
  'thing',
  'storm

In [20]:
id2word=corpora.Dictionary(data_lemmatized)
texts=data_lemmatized  # list of list of tokens
corpus=[id2word.doc2bow(text) for text in texts]
print(corpus[:1])   #first document

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 3), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1)]]


In [21]:
id2word[0]

'advantage'

In [22]:
print([[(id2word[id],freq) for (id,freq) in cp]for cp in corpus[:2]])

[[('advantage', 1), ('area', 1), ('border', 1), ('bridge', 1), ('burial_peace', 1), ('carry', 1), ('command', 1), ('company', 1), ('controversial', 1), ('couple', 1), ('depth', 1), ('even', 3), ('evidence', 1), ('flash', 1), ('get', 1), ('give', 1), ('grave', 1), ('guard', 1), ('helicopter', 1), ('hold', 1), ('image', 2), ('immediately', 1), ('important', 1), ('independent', 1), ('investigation', 1), ('invoke', 1), ('matter', 1), ('peace', 1), ('piece', 1), ('prevent', 1), ('protect', 1), ('public', 1), ('public_domain', 1), ('question', 1), ('relative', 1), ('repeatedly', 1), ('save', 1), ('sea', 1), ('ship', 1), ('sink', 1), ('socalle', 1), ('stormy', 1), ('study', 1), ('theory', 1), ('truth', 1), ('want', 1), ('world', 1)], [('company', 1), ('independent', 1), ('save', 1), ('stormy', 3), ('theory', 1), ('able', 1), ('accident', 1), ('addition', 1), ('amazing', 1), ('argument', 1), ('authority', 1), ('bottom', 1), ('continue', 1), ('demand', 1), ('describe', 1), ('dive', 1), ('excite

# Train

In [23]:
# building LDA Model
lda_model=gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=3,random_state=100,update_every=0,
                                         chunksize=500,passes=20,alpha='auto',per_word_topics=True)


In [24]:
print(lda_model.print_topics())

[(0, '0.003*"get" + 0.002*"car" + 0.002*"law" + 0.001*"re" + 0.001*"thing" + 0.001*"bit_bit" + 0.001*"go" + 0.001*"time" + 0.001*"drive" + 0.001*"man"'), (1, '0.011*"get" + 0.009*"even" + 0.008*"go" + 0.007*"think" + 0.007*"camera" + 0.007*"time" + 0.006*"take" + 0.006*"good" + 0.006*"look" + 0.006*"people"'), (2, '0.012*"get" + 0.010*"money" + 0.009*"go" + 0.008*"time" + 0.007*"even" + 0.007*"camera" + 0.007*"good" + 0.007*"know" + 0.007*"people" + 0.007*"also"')]


In [25]:
doc_lda=lda_model[corpus]
print(list(doc_lda)[0])

([(1, 0.6387385), (2, 0.3612567)], [(0, [1, 2]), (1, [1, 2]), (2, [1, 2]), (3, [1, 2]), (4, [1, 2]), (5, [1, 2]), (6, [1, 2]), (7, [1, 2]), (8, [1, 2]), (9, [1, 2]), (10, [1, 2]), (11, [1, 2]), (12, [1, 2]), (13, [1, 2]), (14, [1, 2]), (15, [1, 2]), (16, [1, 2]), (17, [1, 2]), (18, [1, 2]), (19, [1, 2]), (20, [1, 2]), (21, [1, 2]), (22, [1, 2]), (23, [1, 2]), (24, [1, 2]), (25, [1, 2]), (26, [1, 2]), (27, [1, 2]), (28, [1, 2]), (29, [1, 2]), (30, [1, 2]), (31, [1, 2]), (32, [1, 2]), (33, [1, 2]), (34, [1, 2]), (35, [1, 2]), (36, [1, 2]), (37, [1, 2]), (38, [1, 2]), (39, [1, 2]), (40, [1, 2]), (41, [1, 2]), (42, [1, 2]), (43, [1, 2]), (44, [1, 2]), (45, [1, 2]), (46, [1, 2])], [(0, [(1, 0.6360069), (2, 0.36325082)]), (1, [(1, 0.7483668), (2, 0.2515325)]), (2, [(1, 0.6373401), (2, 0.3622347)]), (3, [(1, 0.6167151), (2, 0.38201997)]), (4, [(1, 0.54920995), (2, 0.3891927)]), (5, [(1, 0.63484925), (2, 0.36492452)]), (6, [(1, 0.59840906), (2, 0.4001938)]), (7, [(1, 0.59357136), (2, 0.4063229

In [26]:
#LDA Model Perfromance check
print("Perplexity: ",lda_model.log_perplexity(corpus))

#compute coherence score
coherence_model_lda=CoherenceModel(model=lda_model,texts=data_lemmatized,dictionary=id2word,coherence='c_v')

coherence_lda=coherence_model_lda.get_coherence()
print('Coherence Score: ',coherence_lda)

Perplexity:  -7.862532441244879
Coherence Score:  0.545079419847477


In [27]:
warnings.filterwarnings('ignore',category=FutureWarning)

pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim.prepare(lda_model,corpus,id2word)
vis

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1     -0.129882  0.0       1        1  63.871356
2     -0.120812  0.0       2        1  36.128159
0      0.250694  0.0       3        1   0.000486, topic_info=         Term          Freq         Total Category  logprob  loglift
14        get  62035.000000  62035.000000  Default  30.0000  30.0000
202       car  24208.000000  24208.000000  Default  29.0000  29.0000
1311      law  11094.000000  11094.000000  Default  28.0000  28.0000
170        re  26740.000000  26740.000000  Default  27.0000  27.0000
80      thing  28928.000000  28928.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
94     happen      0.012966  10334.372615   Topic3  -7.6185  -1.3535
568   finland      0.012518   8905.144672   Topic3  -7.6537  -1.2399
143      know      0.013822  32358.493384   Topic3  -7.5546  -2.4310
116       ill      0.011527   7082.980452   Topic3  -7.7362  -1.0934
171     right      0.011760  17604.320175   Topic3  -7.7162  -1.9838

[299 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
5299      2  0.716157  accle
3274      1  0.773290    aid
3274      2  0.227679    aid
1042      1  0.757406    air
1042      2  0.242708    air
...     ...       ...    ...
601       2  0.353885   work
924       1  0.740254  worth
924       2  0.259631  worth
114       1  0.683216   year
114       2  0.316792   year

[350 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])